##install required dependencies

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

#### Setup Mistral API

In [102]:
from google.colab import userdata
import os
mistral_api_key = userdata.get('MISTRAL_API_KEY')
os.environ["OPENAI_API_KEY"] = mistral_api_key
os.environ["OPENAI_API_BASE"] = "https://api.mistral.ai/v1"
os.environ["OPENAI_MODEL_NAME"] = "mistral-small"
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')



In [ ]:
from langchain_groq import ChatGroq
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [37]:
from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


#### Steps to Get Started


To effectively use the SerperDevTool, follow these steps:

Package Installation: Confirm that the crewai[tools] package is installed in your Python environment.

API Key Acquisition: Acquire a serper.dev API key by registering for a free account at serper.dev.

Environment Configuration: Store your obtained API key in an environment variable named SERPER_API_KEY to facilitate its use by the tool.

## Set up tools

In [3]:
from crewai import Agent,Task,Crew,Process
from crewai_tools import SerperDevTool,ScrapeWebsiteTool,WebsiteSearchTool

In [11]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [12]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
website_search_tool = WebsiteSearchTool()


In [9]:
!pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 29.2 MB/s eta 0:00:00


In [10]:
from langchain_community.tools import DuckDuckGoSearchResults


search = DuckDuckGoSearchResults(num_results=5)
output = search.invoke('comparative study of LangGraph, Autogen and Crewai for building multi-agent system')
print(output)

[snippet: We'll also explore three leading frameworks—AutoGen, CrewAI, and LangGraph—comparing their features, autonomy levels, and ideal use cases, before concluding with strategic recommendations for adopting these frameworks. The Building Blocks of Multi-Agent Systems. Multi-agent systems are akin to a functional team, where each member (agent ..., title: An Overview of Multi Agent Frameworks: Autogen, CrewAI and LangGraph, link: https://sajalsharma.com/posts/overview-multi-agent-fameworks/], [snippet: Multi-agent Collaboration: Crewai's strength lies in its ability to build and manage systems with multiple AI agents working together. ... Provides great flexibility for customization and building ..., title: Langchain vs. Crew.ai vs. Microsoft Autogen: A Comparison - LinkedIn, link: https://www.linkedin.com/pulse/langchain-vs-crewai-microsoft-autogen-comparison-deepak-devassykutty-8awoc], [snippet: 2. Agent with Tools. Now comes the most fun part of CrewAI's agent construction — the 

In [15]:
from langchain.agents import load_tools
@tool("search tool")
def topic_search_tool(topic: str) -> str:
    """Get news for a given cryptocurrency ticker symbol"""
    return search_tool.run(topic)

## Create Agents

#### 1. Context Planner Agent

In [16]:
planner = Agent(role="Conetxt Planner",
                goal = "Plan engaging and factually accurate context on the {topic}",
                backstory="You are working on a planning to write a blogpost about "
                "the topic {topic} in 'https://medium.com/'.You collect information that helps the "
                "audience leran something new and make informed decisions.You have to prepare a detailed "
                "outline and the relevant topics and sub-topics that has to be a part of the"
                "blogpost.Your work is the basis for the Content Writer to write an engaging "
                "and detailed blogpost on the topic provided",
                allow_delegation=False,
                llm=llm,
                max_iter=5,
                # max_rpm=15,
                # memory=True,
                #max_rpm=None, # No limit on requests per minute
                #max_iter=15, # Default value for maximum iterations
                verbose=True)


#### 2. Content Writer Agent

In [25]:
content_writer = Agent(role="Content_Writer",
                       goal="Write insightful and factually accurate"
                       "opinion piece about the topic {topic}",
                       backstory="You are working on a writing to write an opinion piece about "
                       "the topic : {topic}.You base your writing based on the work of the "
                       "Context planer, who provides an outline and relevant context about the topic."
                       "You will follow the main objectives and direction of the outline"
                       ", as provided by the Content Planner.You also provide Objective,impartial "
                       "insights and a meaningful conclusion and bak them up with the information"
                       " provided by the Content Planner.You acknowledge in your opinion piece "
                       " when your statements are opinions as opposed to objective statements",
                       allow_delegation=False,
                       #tools=[website_search_tool],
                       llm=llm,
                       max_iter=5,
                       #max_rpm=15,
                       #memory=True,
                       #max_rpm=None, # No limit on requests per minute
                       #max_iter=15, # Default value for maximum iterations
                       verbose=True,)


#### 3. Content Editor Agent

In [26]:
content_editor= Agent(role="Editor",
                      goal="Edit a given blogpost to align with the writing style ollowed in 'https://medium.com/'.",
                      backstory="You are an editor who recieves a blogpost form the Content Writer."
                      "Your goal is to review the blogpost to ensure that it follows journalistic best practices,"
                      "provides balanced view points when providing opinions or assertions and also "
                      "avoids major controversial topics or opinions when possible.",
                      allow_delegation=False,
                      llm=llm,
                      max_iter=5,
                      #max_rpm=15,
                      #memory=True,
                      #max_rpm=None, # No limit on requests per minute
                      #max_iter=15, # Default value for maximum iterations
                      verbose=True)

In [253]:
content_editor

Agent(role=Editor, goal=Edit a given blogpost to align with the writing style ollowed in 'https://medium.com/'., backstory=You are an editor who recieves a blogpost form the Content Writer.Your goal is to review the blogpost to ensure that it follows journalistic best practices,provides balanced view points when providing opinions or assertions and also avoids major controversial topics or opinions when possible.)

## Create a task for each agent

In [27]:
content_planner_task = Task(description=(
    "1. Prioritize the latest trends,key players and noteworthy news on {topic}\n"
    "2. Identify the targt audience,considering their interests and pain points"
    "3. Develop a detailed content outline including an introduction,key points,a call to actions, sample code if applicable.\n"
    "4. Include SEO keywords and relevant data or sources use dfor reffrence"),
    expected_output = "A comprehenive content plan documen with an outline,audience analysis,"
    "SEO keywords,Conclusion,referrence resources.",
    agent=planner,
    verbose=True

)

In [28]:
writer_task = Task(description=(
    "1. Use the content plan to craft a compelling blogpost on{topic}"
    "2. Incorporate SEO keywords naturally."
    "3. Sections or Subtitles are properly named in an engaging manner."
    "4. Ensure the post is structured with an engaging introduction,insightful body and a summarizing conclusion."
    "5. Proofread for grammatical errors and alignment with the brand's voice."
    ),
    expected_output = "A well written blogspost in markdown format,ready for publication,"
    "each section should have 2 or 3 paragraphs.",
    agent=content_writer,
    verbose=True

)

In [29]:
editor = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    output_file="blogpost.md",
    agent=content_editor,
    verbose=True,
)

## Build the Crew

In [30]:
crew = Crew(agent=[planner,content_writer,content_editor],
    tasks=[content_planner_task,writer_task,editor],
            verbose=2,
            process=Process.sequential,
            full_output=True,
            share_crew=False,
            manager_llm=llm,
            max_iter=15,
            )

##Formulate the imnput

In [31]:
inputs = {"topic":"Comparative study of LangGraph, Autogen and Crewai for building multi-agent system"}

In [33]:
inputs = {"topic":"How to build a multi-agent system."}

## Execute the multiagent System

In [36]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Conetxt Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends,key players and noteworthy news on How to build a multi-agent system.
2. Identify the targt audience,considering their interests and pain points3. Develop a detailed content outline including an introduction,key points,a call to actions, sample code if applicable.
4. Include SEO keywords and relevant data or sources use dfor reffrence


> Entering new CrewAgentExecutor chain...
I now can give a great answer!

Final Answer:

**Content Plan Document: How to Build a Multi-Agent System**

**Target Audience:**

* Software developers and engineers interested in artificial intelligence and distributed systems
* Researchers and academics in the field of multi-agent systems
* Businesses and organizations looking to implement multi-agent systems in their operations

**Interests and Pain Points:**

* Understanding the basics of multi-agent systems and their applications
* Learning how to design 

IndexError: list index out of range